In [ ]:
!pip install pymilvus milvus tiktoken openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
data = pd.read_excel('/content/dataset 2.xlsx',header=None, names=['question', 'answer'])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  98 non-null     object
 1   answer    98 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [ ]:
# Question 1
new_data_ml = {'question': "What is machine learning?",
               'answer': "Machine learning is a field of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computer systems to perform tasks without explicit programming. It involves the use of data and iterative learning to improve the system's performance over time."
}
# Adding the data to the existing dataset
data = data.append(new_data_ml, ignore_index=True)

# Question 2
new_data_ai = {'question': "What is AI?",
               'answer': "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and mimic human cognitive functions. These systems can perform tasks such as problem-solving, learning, understanding natural language, and adapting to new information, aiming to replicate intelligent behavior."
}
# Adding the data to the existing dataset
data = data.append(new_data_ai, ignore_index=True)

# Checking information about the dataset
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  100 non-null    object
 1   answer    100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


<ipython-input-3-ac0ff228558c>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_data_ml, ignore_index=True)
<ipython-input-3-ac0ff228558c>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_data_ai, ignore_index=True)


## Word2vec

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

#for model prep
tokenized_texts = [word_tokenize(text.lower()) for text in (list(data['answer']) + list(data['question']))]
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=300, window=5, min_count=1, workers=4)

# Get embeddings for each text
tokenized_texts = [word_tokenize(text.lower()) for text in data['answer']]
w2v_embeddings = [word2vec_model.wv[words].mean(axis=0) for words in tokenized_texts]

print('No of sentence embeddings:', len(w2v_embeddings))
print('Dim size:', len(w2v_embeddings[0]))

No of sentence embeddings: 100
Dim size: 300


In [ ]:
tokenized_texts = [word_tokenize(text.lower()) for text in data['question']]
word2vec_question_vectors = [word2vec_model.wv[words].mean(axis=0) for words in tokenized_texts]
print('No of sentence embeddings:', len(word2vec_question_vectors))
print('Dim size:', len(word2vec_question_vectors[0]))

No of sentence embeddings: 100
Dim size: 300


In [ ]:
ids = [n for n, x in enumerate(w2v_embeddings)]
import pandas as pd
word2vec = pd.DataFrame({
    'ids':ids,
    'vectors': w2v_embeddings,
    'question': data['question'],
    'answer':data['answer'],
    'question_vector':word2vec_question_vectors
})
word2vec.to_json('word2vec.json')
word2vec.head()

,ids,vectors,question,answer,question_vector
0,0,"[-0.012808545, 0.03545821, 0.0060042376, 0.026...",What is Artificial Intelligence (AI)?,Artificial Intelligence (AI) refers to the sim...,"[-0.010552682, 0.025525333, 0.0038705692, 0.01..."
1,1,"[-0.01118819, 0.029990787, 0.0055663483, 0.022...",How does Machine Learning differ from traditio...,"In traditional programming, explicit instructi...","[-0.008278496, 0.023763793, 0.0036814047, 0.01..."
2,2,"[-0.011429683, 0.03021969, 0.0055796653, 0.022...",What are the different types of Machine Learning?,Machine Learning is broadly categorized into t...,"[-0.011084137, 0.030802922, 0.005074119, 0.022..."
3,3,"[-0.0128287515, 0.03470946, 0.0061058914, 0.02...",Explain Natural Language Processing (NLP).,Natural Language Processing (NLP) is a branch ...,"[-0.0091219405, 0.0264889, 0.0035194294, 0.019..."
4,4,"[-0.010972189, 0.030116104, 0.0054813814, 0.02...",What is the role of Neural Networks in AI?,Neural Networks are a fundamental component of...,"[-0.012371021, 0.034045078, 0.006060354, 0.025..."


mlvius

In [ ]:
import json, time
from pymilvus import MilvusClient

CLUSTER_ENDPOINT="https://in03-3f134871b23989a.api.gcp-us-west1.zillizcloud.com" # Set your cluster endpoint
TOKEN="a8edd7739aea672a91f15255b530d61089322666f5af4bee08668221af89037abcbd7154f69615b07a0f5cb202c6ddd060237d83" # Set your token
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 073c71ae06c14556921ffa396d4e5a6a


In [ ]:
COLLECTION_NAME = 'word2vec'

client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=300
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: word2vec
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: word2vec


In [ ]:
res = client.describe_collection(
    collection_name=COLLECTION_NAME
)

print(res)

{'collection_name': 'word2vec', 'auto_id': False, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': 5, 'params': {}, 'element_type': 0, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': '', 'type': 101, 'params': {'dim': 300}, 'element_type': 0}], 'aliases': [], 'collection_id': 446011777549528245, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}


In [ ]:
for n, vector in enumerate(tqdm(w2v_embeddings)):
  res = client.insert(
    collection_name=COLLECTION_NAME,
    data={
        'id':n,
        'vector': vector
        }
    )

100%|██████████| 100/100 [00:11<00:00,  8.80it/s]


Search

In [ ]:
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[]
    )
id = res[0][0]['id']
bert['text'][id]

# Glove

In [ ]:
!python -m spacy download en_core_web_md
import spacy

nlp = spacy.load('en_core_web_md')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Generate sentence embeddings for answers
glove_embeddings_answers = [sent.vector for sent in nlp.pipe(data['answer'])]

# Generate sentence embeddings for questions
glove_embeddings_questions = [sent.vector for sent in nlp.pipe(data['question'])]

# Print information about the embeddings
print('No of sentence embeddings for answers:', len(glove_embeddings_answers))
print('Dim size:', len(glove_embeddings_answers[0]))

print('No of sentence embeddings for questions:', len(glove_embeddings_questions))
print('Dim size:', len(glove_embeddings_questions[0]))

No of sentence embeddings for answers: 100
Dim size: 300
No of sentence embeddings for questions: 100
Dim size: 300


In [ ]:
ids = [n for n, x in enumerate(w2v_embeddings)]
import pandas as pd
word2vec = pd.DataFrame({
    'ids':ids,
    'vectors': glove_embeddings_answers,
    'question': data['question'],
    'answer':data['answer'],
    'question_vector':glove_embeddings_questions
})
word2vec.to_json('glove.json')
word2vec.head()

,ids,vectors,question,answer,question_vector
0,0,"[-1.9772749, -0.03724464, -0.90389884, 0.35585...",What is Artificial Intelligence (AI)?,Artificial Intelligence (AI) refers to the sim...,"[-3.0649638, 0.5506875, 4.242112, 0.8929238, -..."
1,1,"[-1.7680229, 0.39258194, -0.9004436, 0.1482030...",How does Machine Learning differ from traditio...,"In traditional programming, explicit instructi...","[-1.2723067, 0.3909554, -1.4864812, 0.21192776..."
2,2,"[-1.7309706, -0.6482432, -1.0639645, 1.1340513...",What are the different types of Machine Learning?,Machine Learning is broadly categorized into t...,"[-4.46414, -0.8125167, -1.1054968, 2.128201, 4..."
3,3,"[-1.9882762, -0.46295467, 0.48355773, -0.16925...",Explain Natural Language Processing (NLP).,Natural Language Processing (NLP) is a branch ...,"[-2.9807355, -4.290172, 5.2013526, 0.44720376,..."
4,4,"[-2.8926997, -1.1227449, -0.44649503, 1.091930...",What is the role of Neural Networks in AI?,Neural Networks are a fundamental component of...,"[-3.368321, 3.242681, 0.412426, 0.60325456, 5...."


In [ ]:
COLLECTION_NAME = 'glove'

client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=300
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: glove
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: glove


In [ ]:
for n, vector in enumerate(tqdm(glove_embeddings_answers)):
  res = client.insert(
    collection_name=COLLECTION_NAME,
    data={
        'id':n,
        'vector': vector
        }
    )

100%|██████████| 100/100 [00:11<00:00,  8.77it/s]


## BERT

In [ ]:

import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Lists to store BERT embeddings
embeddings_bert_answers = []
embeddings_bert_questions = []

batch_size = 5  # Set your desired batch size

quesion_answer = [x+':\n'+y for x,y in zip(data['question'], data['answer'])]

# Use tqdm to add a progress bar
for i in tqdm(range(0, len(data['answer']), batch_size), desc="Processing BERT Embeddings for Answers"):
    batch_sentences = quesion_answer[i:i + batch_size]
    inputs = tokenizer(list(batch_sentences), return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    batch_embeddings = outputs.pooler_output.detach().numpy()
    embeddings_bert_answers.extend(batch_embeddings)

for i in tqdm(range(0, len(data['question']), batch_size), desc="Processing BERT Embeddings for Questions"):
    batch_sentences = data['question'][i:i + batch_size]
    inputs = tokenizer(list(batch_sentences), return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    batch_embeddings = outputs.pooler_output.detach().numpy()
    embeddings_bert_questions.extend(batch_embeddings)

# Print or use the embeddings for downstream tasks
print('No of sentence embeddings for answers:', len(embeddings_bert_answers))
print('Dim size:', len(embeddings_bert_answers[0]))

print('No of sentence embeddings for questions:', len(embeddings_bert_questions))
print('Dim size:', len(embeddings_bert_questions[0]))

Processing BERT Embeddings for Questions: 100%|██████████| 20/20 [00:08<00:00,  2.29it/s]

No of sentence embeddings for answers: 100
Dim size: 768
No of sentence embeddings for questions: 100
Dim size: 768


In [ ]:
quesion_answer[99]

'What is AI?:\nArtificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and mimic human cognitive functions. These systems can perform tasks such as problem-solving, learning, understanding natural language, and adapting to new information, aiming to replicate intelligent behavior.'

In [ ]:
ids = [n for n, x in enumerate(embeddings_bert_answers)]
import pandas as pd
bert = pd.DataFrame({
    'ids':ids,
    'vectors': embeddings_bert_answers,
    'question': data['question'],
    'answer':data['answer'],
    'question_vector':embeddings_bert_questions
})
bert.to_json('bert.json')
bert.head()

,ids,vectors,question,answer,question_vector
0,0,"[-0.8494552, -0.54522204, -0.94483817, 0.18418...",What is photosynthesis and its significance in...,Photosynthesis is the process by which green p...,"[-0.9286332, -0.43703884, -0.9002013, 0.791641..."
1,1,"[-0.8208226, -0.5115974, -0.936001, 0.2516346,...",How does DNA replication occur in cells?,DNA replication is the process where a cell du...,"[-0.9308114, -0.5421242, -0.9354812, 0.8387334..."
2,2,"[-0.693108, -0.5847721, -0.9791001, 0.19533502...",What is the role of mitochondria in cellular f...,Mitochondria are organelles known as the power...,"[-0.9406521, -0.5694683, -0.9525867, 0.8627299..."
3,3,"[-0.7020544, -0.39628774, -0.92711484, 0.11913...",How do enzymes function in biological systems?,Enzymes are biological catalysts that accelera...,"[-0.897881, -0.44796792, -0.8573425, 0.7368364..."
4,4,"[-0.8299607, -0.6210202, -0.959827, 0.33729538...",What are stem cells and their potential applic...,Stem cells are undifferentiated cells with the...,"[-0.90765893, -0.47947496, -0.89814615, 0.7841..."


In [ ]:
import json, time
from pymilvus import MilvusClient

CLUSTER_ENDPOINT="https://in03-e0cdc40beba89a7.api.gcp-us-west1.zillizcloud.com" # Set your cluster endpoint
TOKEN="902a1a6f6d7bb11afb1464036bf1415c75fda549f14e93de6a15eabec565bf9cf6ec28611825ff65910cc573f8470a2acd171c14" # Set your token
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: e750e6a63ae04a40aa532c2154a156e6


In [ ]:
COLLECTION_NAME = 'hf'

client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=384
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: hf
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: hf


In [ ]:
len(embeddings_bert_answers)
embeddings_bert_answers[0].shape

(768,)

In [ ]:
for n, vector in enumerate(tqdm(embeddings_bert_answers)):
  res = client.insert(
    collection_name=COLLECTION_NAME,
    data={
        'id':n,
        'vector': vector
        }
    )

100%|██████████| 100/100 [00:21<00:00,  4.68it/s]


In [ ]:
def bert_query(query):
  input = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
  outputs = model(**input)
  return outputs.pooler_output.detach().numpy()

# bert_query_vector= bert_query(query)

In [ ]:
embeddings_bert_questions[0].shape

(768,)

In [ ]:
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[embeddings_bert_questions[0]]
    )
id1 = res[0][0]['id']
id2 = res[0][1]['id']
id3 = res[0][2]['id']

print(id1,id2,id3)

55 43 38


In [ ]:
query = input('Enter: ')
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[bert_query(query)[0]]
    )
id1 = res[0][0]['id']
id2 = res[0][1]['id']
id3 = res[0][2]['id']
bert['answer'][id1]
print(bert['answer'][id1])
print(bert['answer'][id2])
print(bert['answer'][id2])

Enter: 
The Russian Revolution of 1917 was sparked by a culmination of factors including social discontent, economic hardships, and the consequences of World War I. It began with the February Revolution, resulting in the overthrow of Tsar Nicholas II and the establishment of a provisional government. However, the Bolsheviks, led by Vladimir Lenin, seized power during the October Revolution, establishing a communist government. The revolution led to civil war between the Bolsheviks (Reds) and their opponents (Whites). The consequences included the establishment of the Soviet Union, the redistribution of land, nationalization of industries, and the execution of the royal family. The revolution profoundly transformed Russia's social, political, and economic structures.
The Protestant Reformation, launched by Martin Luther in 1517, challenged the authority of the Roman Catholic Church and led to significant religious and social transformations. Luther's Ninety-Five Theses criticized church

In [ ]:
id=[]
for n,x in enumerate(embeddings_bert_questions):
  res = client.search(
    collection_name=COLLECTION_NAME,
    data=[embeddings_bert_questions[n]]
    )
  id.append(res[0][0]['id'])

In [ ]:
accuracy_score(id, [x for x in range(0,100)])

0.01

In [ ]:
!pip install faiss-cpu sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00


## using faiss

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(768)   # build the index, d=size of vectors
#IndexFlatL2
#IndexFlatIP
# here we assume xb contains a n-by-d numpy matrix of type float32
index.add(np.array(embeddings_bert_answers))

In [ ]:
# xq is a n2-by-d matrix with query vectors
k = 4                          # we want 4 similar vectors
D, I = index.search(np.array([embeddings_bert_questions[0]]), k)     # actual search
print(I)

[[30 73 64 54]]


In [ ]:
query = input('Enter: ')

D, I = index.search(np.array([bert_query(query)[0]]), 4)     # actual search
print(bert['answer'][I[0]])

Enter: What is Artificial Intelligence?
30    World War I was primarily caused by militarism...
73    Torque, represented as force multiplied by the...
50    The American Civil War erupted primarily due t...
97    Nuclear fusion is the process where atomic nuc...
Name: answer, dtype: object


OPENAI

In [ ]:

from openai import OpenAI

client = OpenAI(api_key=api_key)

# openai.OPENAI_API_KEY = api_key
response = client.embeddings.create(
    input='hi my name is zuhair i love ruthwik',
    model="text-embedding-ada-002"
)

print(response.data[0].embedding)

[-0.02300882712006569, 0.012034354731440544, -0.011021429672837257, -0.030776822939515114, -0.008163773454725742, 0.04019501432776451, -0.00829793605953455, -0.027530096471309662, 0.001084198709577322, -0.02714102528989315, 0.007801535539329052, -0.004745990037918091, -0.0009366202866658568, -0.03284292295575142, -0.008982162922620773, -0.012537463568150997, 0.01152453850954771, -0.0006712305475957692, 0.0015520896995440125, 0.0017558485269546509, -0.007291718851774931, 0.010800061747431755, 0.009733472019433975, -0.005517423152923584, -0.018514391034841537, 0.006161401979625225, 0.012175224721431732, 0.0064431424252688885, -0.012886284850537777, -0.03085731901228428, -0.0029532467015087605, 0.012846036814153194, -0.006490099243819714, -0.03158179670572281, -0.029569361358880997, -0.0007680789567530155, 0.007412464823573828, -0.01927911676466465, -0.001304727979004383, 0.01213497668504715, 0.0026715060230344534, -0.009981672279536724, 0.006080904509872198, 0.0046755545772612095, -0.029

In [ ]:
# List to store embeddings
embeddings = []
quesion_answer = [x + ':\n' + y for x, y in zip(data['question'], data['answer'])]

# Loop through each sentence
for n, sentence in enumerate(tqdm(quesion_answer)):
    # Send the sentence to the Text Embedding API
    response = client.embeddings.create(
        input=sentence,
        model="text-embedding-ada-002"
    )
    # Extract the embedding from the API response and append to the list
    embeddings.append(response.data[0].embedding)

    # Add a delay of 20 seconds after every 3 requests
    if (n + 1) % 3 == 0 and n > 0:
      time.sleep(25)

# Split the embeddings into answers and questions
glove_embeddings_answers = embeddings

# ---------------------------------------------------------------------------

# List to store embeddings
embeddings = []
quesion_answer = [x + ':\n' + y for x, y in zip(data['question'], data['answer'])]

# Loop through each sentence
for n, sentence in enumerate(tqdm(quesion_answer)):
    # Send the sentence to the Text Embedding API
    response = client.embeddings.create(
        input=sentence,
        model="text-embedding-ada-002"
    )
    # Extract the embedding from the API response and append to the list
    embeddings.append(response.data[0].embedding)

    # Add a delay of 20 seconds after every 3 requests
    if (n + 1) % 3 == 0 and n > 0:
      time.sleep(25)

glove_embeddings_questions = embeddings

# Print information about the embeddings
print('No of sentence embeddings for answers:', len(glove_embeddings_answers))
print('Dim size:', len(glove_embeddings_answers[0]))

print('No of sentence embeddings for questions:', len(glove_embeddings_questions))
print('Dim size:', len(glove_embeddings_questions[0]))


  4%|▍         | 4/100 [00:28<11:26,  7.15s/it]


RateLimitError: ignored

In [ ]:
!pip install langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
embedding = hf.embed_query("hi this is harrison")


In [ ]:
# Generate sentence embeddings for answers
glove_embeddings_answers = [hf.embed_query(x) for x in data['answer']]

# Generate sentence embeddings for questions
glove_embeddings_questions = [hf.embed_query(x) for x in data['question']]

# Print information about the embeddings
print('No of sentence embeddings for answers:', len(glove_embeddings_answers))
print('Dim size:', len(glove_embeddings_answers[0]))

print('No of sentence embeddings for questions:', len(glove_embeddings_questions))
print('Dim size:', len(glove_embeddings_questions[0]))

No of sentence embeddings for answers: 100
Dim size: 384
No of sentence embeddings for questions: 100
Dim size: 384


In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(384)   # build the index, d=size of vectors
#IndexFlatL2
#IndexFlatIP
# here we assume xb contains a n-by-d numpy matrix of type float32
index.add(np.array(glove_embeddings_answers))

In [ ]:
k = 4                          # we want 4 similar vectors
D, I = index.search(np.array([glove_embeddings_questions[96]]), k)
print(I[0][0])

96


In [ ]:
glove_embeddings_questions[0]

AttributeError: ignored

In [ ]:
k = 4                          # we want 4 similar vectors
D, I = index.search(np.array([glove_embeddings_questions[0]]), k)     # actual search
print(I)

[[ 0 24  2 28]]


In [ ]:
query = input('Enter: ')

D, I = index.search(np.array([hf.embed_query(query)]), k)     # actual search
print(bert['answer'][I[0]])

Enter: 
19    Hormones are chemical messengers produced by e...
21    Osmosis is the movement of water across a sele...
0     Photosynthesis is the process by which green p...
3     Enzymes are biological catalysts that accelera...
Name: answer, dtype: object


All aside from scratch

In [ ]:
# Generate sentence embeddings for answers
hf_embeddings_answers = [hf.embed_query(x) for x in data['answer']]

# Generate sentence embeddings for questions
hf_embeddings_quesions = [hf.embed_query(x) for x in data['answer']]

# Print information about the embeddings
print('No of sentence embeddings for answers:', len(hf_embeddings_answers))
print('Dim size:', len(hf_embeddings_answers[0]))

print('No of sentence embeddings for questions:', len(hf_embeddings_quesions))
print('Dim size:', len(hf_embeddings_quesions[0]))

No of sentence embeddings for answers: 100
Dim size: 384
No of sentence embeddings for questions: 100
Dim size: 384


In [ ]:
import faiss
index = faiss.IndexFlatL2(384)
index.add(np.array(hf_embeddings_answers))

In [ ]:
k = 4                          # we want 4 similar vectors
D, I = index.search(np.array([hf_embeddings_quesions[96]]), k)
print(I[0][0])

96


In [ ]:
query = str(input('Enter: '))

D, I = index.search(np.array([hf.embed_query(query)]), k)     # actual search
n = I[0][0]
print(data['answer'][int(n)])

Enter: what is machine learning?
Machine learning is a field of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computer systems to perform tasks without explicit programming. It involves the use of data and iterative learning to improve the system's performance over time.


In [ ]:
for n, vector in enumerate(tqdm(hf_embeddings_answers)):
  res = client.insert(
    collection_name=COLLECTION_NAME,
    data={
        'id':n,
        'vector': vector
        }
    )

100%|██████████| 100/100 [00:21<00:00,  4.71it/s]


In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import numpy as np
# Streamlit app
st.title("Simple AI Search App")

query = st.text_input("Enter your question:")
if query:
    # Perform vectorized search
    D, I = index.search(np.array([hf.embed_query(query)]), k=1)
    n = I[0][0]

    # Display the answer
    st.subheader("Answer:")
    st.write(data['answer'][int(n)])

2023-12-04 06:48:17.891 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.148.176:8501

  Stopping...
  Stopping...


In [ ]:
%%bash
hostname -I

172.28.0.12 
